# 📊 PyPortfolio - Otimização de Portfólio v2.7**Markowitz, Fronteira Eficiente, Monte Carlo e Sharpe vs CDI**---## 📋 Metadados da Versão| Campo | Valor ||-------|-------|| **Versão** | v2.7 || **Data** | Janeiro/2026 || **Autor** | PyPortfolio Team |### Changelog v2.7- ✅ **P0: Conversão USD→BRL** - Retornos de ativos em USD convertidos para BRL usando série de câmbio- ✅ **P0: Metadados por ativo** - Estrutura com moeda, fonte e ticker- ✅ **P0: Política de peso fora do universo** - 3 opções configuráveis (reportar/renormalizar/caixa CDI)- ✅ **P1: Integração yfinance opcional** - Fallback para ativos sem dados no Excel- ✅ **P1: Cache local** - Dados baixados são salvos para reutilização- ✅ **Validação de conversão FX** - Bloco de auditoria mostrando cálculo r_BRL### Mudanças desde v2.6:- Estrutura de metadados por ativo (moeda/fonte/ticker)- Download de série de câmbio USD/BRL via yfinance- Fórmula r_BRL = (1+r_USD)*(1+r_FX) - 1 implementada- Hierarquia de fontes: Excel → yfinance → DEFAULT_UNIVERSE- Auditoria detalhada de moeda e cobertura

---## 📖 Parâmetros e Configuração (Documentação)### Novos Parâmetros v2.7| Parâmetro | Valor Padrão | O que controla | Quando ajustar | Impacto / Trade-off ||-----------|--------------|----------------|----------------|---------------------|| `ATIVOS_USD` | ["GOOGL", "NVDA", ...] | Lista de ativos cuja moeda é USD | Se adicionar/remover ativos americanos | Conversão FX só é aplicada a estes ativos || `FX_TICKER` | "USDBRL=X" | Ticker do câmbio USD/BRL no Yahoo Finance | Se o ticker mudar | Série errada = conversão incorreta || `USAR_YFINANCE` | True | Se permite fallback para yfinance | Se quer usar apenas Excel | True: mais robusto. False: só Excel || `CACHE_DIR` | "cache/" | Pasta para cache de dados baixados | Se quer mudar local do cache | - || `CACHE_DIAS_VALIDADE` | 7 | Dias até o cache expirar | Se quer dados mais/menos frescos | Menor: mais downloads. Maior: dados potencialmente desatualizados || `POLITICA_PESO_FORA` | "reportar" | O que fazer com peso fora do universo | Conforme estratégia | "reportar": só avisa. "renormalizar": ajusta para 100%. "caixa_cdi": trata como RF |### Parâmetros de Qualidade de Dados| Parâmetro | Valor Padrão | O que controla | Quando ajustar | Impacto / Trade-off ||-----------|--------------|----------------|----------------|---------------------|| `MIN_OBS` | 12 | Mínimo de meses de histórico por ativo | Se ativos novos estão sendo excluídos | Valor baixo: mais ruído. Valor alto: exclui ativos novos || `MIN_OVERLAP` | 12 | Mínimo de meses em comum para covariância | Se muitas covariâncias estão zeradas | Valor baixo: menos confiável. Valor alto: matriz esparsa || `VOL_FLOOR_MENSAL` | 0.005 | Piso de volatilidade mensal | Se RF domina GMV | Evita vol artificial ~0 |### Parâmetros de Mapeamento| Parâmetro | Valor Padrão | O que controla | Quando ajustar | Impacto / Trade-off ||-----------|--------------|----------------|----------------|---------------------|| `THRESHOLD_OK` | 0.90 | Score para aceitar automaticamente | Se mapeamentos corretos são rejeitados | Baixo: falsos positivos || `THRESHOLD_REVISAR` | 0.75 | Score para aceitar com flag | Se muitos não mapeados | Baixo: matches ruins || `ATIVOS_EXCLUIR` | ["USDC"] | Lista de exclusão manual | Stablecoins distorcem GMV | - |### Parâmetros de Monte Carlo| Parâmetro | Valor Padrão | O que controla ||-----------|--------------|----------------|| `NUM_PORTFOLIOS` | 80000 | Quantidade de simulações || `MC_ALPHA_CONC` | 0.2 | Dirichlet concentrado || `MC_ALPHA_DIV` | 1.0 | Dirichlet diversificado || `MC_FRAC_CANTOS` | 0.05 | Fração quase-100% em 1 ativo || `MC_FRAC_SPARSE` | 0.15 | Fração sparse (3-8 ativos) |### Parâmetros de Otimização| Parâmetro | Valor Padrão | O que controla ||-----------|--------------|----------------|| `PESO_MIN_ATIVO` | 0.0 | Peso mínimo (long-only) || `PESO_MAX_ATIVO` | 1.0 | Peso máximo por ativo || `N_PONTOS_FRONTEIRA` | 60 | Pontos na fronteira eficiente || `RF_MODO` | "media_periodo" | Modo de obtenção do CDI |

---## 🔧 BLOCO 0 - CONFIGURAÇÕES GLOBAIS

In [ ]:
# =============================================================================# METADADOS DA VERSÃO# =============================================================================VERSAO = "v2.7"DATA_VERSAO = "Janeiro/2026"MUDANCAS_VERSAO = [    "P0: Conversão USD→BRL com série de câmbio",    "P0: Metadados por ativo (moeda/fonte/ticker)",    "P0: Política configurável para peso fora do universo",    "P1: Integração opcional com yfinance + cache",    "P1: Validação de conversão FX com exemplos",    "Auditoria detalhada de moeda e cobertura"]print(f"📊 PyPortfolio {VERSAO} ({DATA_VERSAO})")print("   Mudanças nesta versão:")for m in MUDANCAS_VERSAO:    print(f"   • {m}")

In [ ]:
# =============================================================================# CONFIGURAÇÕES GLOBAIS - EDITE AQUI# =============================================================================# --- Caminhos de Arquivos ---EXCEL_DIR = "1 - Dados/1 - Rentabilidade atual"EXCEL_PATTERN = "*.xlsx"ARQUIVO_PADRAO = "Rendimentos_Mensais_Ativos_v7.0.xlsx"# --- Cache e yfinance ---USAR_YFINANCE = True           # Permite fallback para yfinanceCACHE_DIR = "cache/"           # Pasta para cacheCACHE_DIAS_VALIDADE = 7        # Dias até expirar# --- Ativos em USD (serão convertidos para BRL) ---# IMPORTANTE: Liste aqui todos os ativos cujos retornos estão em USDATIVOS_USD = [    "GOOGL", "NVDA", "AAPL", "MSFT", "AMZN", "META", "TSLA",    "BTC", "ETH", "SOL",  # Cripto geralmente cotado em USD    # Adicione outros conforme necessário]# --- Câmbio ---FX_TICKER = "USDBRL=X"         # Ticker USD/BRL no Yahoo FinanceFX_FALLBACK_TICKER = "BRL=X"   # Ticker alternativo# --- Política para peso fora do universo ---# "reportar": apenas reporta (padrão)# "renormalizar": renormaliza para 100% (com aviso)# "caixa_cdi": trata como Caixa CDI fora do ΣPOLITICA_PESO_FORA = "reportar"# --- Parâmetros de Qualidade de Dados ---MIN_OBS = 12MIN_OVERLAP = 12VOL_FLOOR_MENSAL = 0.005# --- Ativos a excluir ---ATIVOS_EXCLUIR = ["USDC"]# --- Thresholds de mapeamento ---THRESHOLD_OK = 0.90THRESHOLD_REVISAR = 0.75ALERTA_COBERTURA_MINIMA = 0.60# --- Taxa Livre de Risco ---RF_MODO = "media_periodo"RF_MANUAL = 0.1150RF_FALLBACK = 0.10# --- Otimização ---PESO_MAX_ATIVO = 1.0PESO_MIN_ATIVO = 0.0# --- Monte Carlo ---NUM_PORTFOLIOS = 80000RANDOM_SEED = 42MC_ALPHA_CONC = 0.2MC_ALPHA_DIV = 1.0MC_FRAC_CANTOS = 0.05MC_FRAC_SPARSE = 0.15# --- Fronteira ---N_PONTOS_FRONTEIRA = 60# --- Retorno ---USAR_MEDIA_GEOMETRICA = True# --- Visualização ---TOP_N_PESOS = 20# --- DEFAULT_UNIVERSE (usado se não houver Excel nem carteira) ---DEFAULT_UNIVERSE = [    "PETR4.SA", "VALE3.SA", "ITUB4.SA", "BBDC4.SA", "ABEV3.SA",    "WEGE3.SA", "RENT3.SA", "EQTL3.SA", "RADL3.SA", "BBAS3.SA"]print("\n✅ Configurações carregadas!")print(f"   Ativos USD: {len(ATIVOS_USD)} | FX: {FX_TICKER}")print(f"   Política peso fora: {POLITICA_PESO_FORA}")print(f"   yfinance: {'Habilitado' if USAR_YFINANCE else 'Desabilitado'}")

---## 📦 BLOCO 1 - BIBLIOTECAS E FUNÇÕES UTILITÁRIAS

In [ ]:
# =============================================================================# IMPORTS# =============================================================================import pandas as pdimport numpy as npfrom scipy.optimize import minimizefrom scipy.linalg import eighimport plotly.graph_objects as goimport warningsfrom pathlib import Pathfrom difflib import SequenceMatcherimport reimport osfrom typing import Optional, Dict, Tuple, List, Anyfrom IPython.display import displayfrom datetime import datetime, timedeltaimport jsontry:    import yfinance as yf    YFINANCE_DISPONIVEL = Trueexcept ImportError:    YFINANCE_DISPONIVEL = False    print("⚠️ yfinance não instalado. Fallback desabilitado.")warnings.filterwarnings('ignore', category=FutureWarning)pd.set_option('display.max_columns', 20)pd.set_option('display.max_rows', 50)print("✅ Bibliotecas importadas!")print(f"   yfinance disponível: {YFINANCE_DISPONIVEL}")

In [ ]:
# =============================================================================# FUNÇÕES - CACHE LOCAL# =============================================================================def garantir_pasta_cache(cache_dir: str) -> Path:    """Cria pasta de cache se não existir."""    path = Path(cache_dir)    path.mkdir(parents=True, exist_ok=True)    return pathdef cache_valido(cache_path: Path, dias_validade: int) -> bool:    """Verifica se cache existe e está válido."""    if not cache_path.exists():        return False    mtime = datetime.fromtimestamp(cache_path.stat().st_mtime)    return datetime.now() - mtime < timedelta(days=dias_validade)def salvar_cache(dados: pd.DataFrame, cache_path: Path):    """Salva DataFrame em cache (parquet)."""    dados.to_parquet(cache_path)    print(f"   💾 Cache salvo: {cache_path.name}")def carregar_cache(cache_path: Path) -> pd.DataFrame:    """Carrega DataFrame do cache."""    df = pd.read_parquet(cache_path)    print(f"   📂 Cache carregado: {cache_path.name}")    return dfprint("✅ Funções de cache carregadas!")

In [ ]:
# =============================================================================# FUNÇÕES - YFINANCE E CÂMBIO# =============================================================================def baixar_serie_yfinance(ticker: str, data_inicio: str, data_fim: str,                           cache_dir: str = None, cache_dias: int = 7) -> pd.Series:    """    Baixa série de preços ajustados do Yahoo Finance.    Usa cache se disponível e válido.    """    if not YFINANCE_DISPONIVEL:        raise ImportError("yfinance não está instalado")        # Verificar cache    if cache_dir:        cache_path = garantir_pasta_cache(cache_dir) / f"{ticker.replace('=', '_')}_prices.parquet"        if cache_valido(cache_path, cache_dias):            df = carregar_cache(cache_path)            return df['Adj Close'] if 'Adj Close' in df.columns else df.iloc[:, 0]        # Baixar dados    print(f"   📥 Baixando {ticker} do Yahoo Finance...")    try:        dados = yf.download(ticker, start=data_inicio, end=data_fim, progress=False)        if dados.empty:            raise ValueError(f"Nenhum dado retornado para {ticker}")                # Salvar cache        if cache_dir:            salvar_cache(dados, cache_path)                return dados['Adj Close']    except Exception as e:        print(f"   ❌ Erro ao baixar {ticker}: {e}")        return pd.Series(dtype=float)def baixar_cambio_usdbrl(data_inicio: str, data_fim: str,                          ticker: str = "USDBRL=X",                         cache_dir: str = None, cache_dias: int = 7) -> pd.Series:    """    Baixa série de câmbio USD/BRL.    Retorna série de preços diários.    """    serie = baixar_serie_yfinance(ticker, data_inicio, data_fim, cache_dir, cache_dias)        if serie.empty:        # Tentar ticker alternativo        print(f"   ⚠️ Tentando ticker alternativo: BRL=X")        serie = baixar_serie_yfinance("BRL=X", data_inicio, data_fim, cache_dir, cache_dias)        return seriedef calcular_retornos_mensais(precos: pd.Series) -> pd.Series:    """    Converte série de preços diários em retornos mensais.    Usa último dia do mês.    """    if precos.empty:        return pd.Series(dtype=float)        # Reamostrar para mensal (último dia)    precos_mensais = precos.resample('M').last()        # Calcular retornos    retornos = precos_mensais.pct_change().dropna()        # Ajustar índice para primeiro dia do mês (consistência)    retornos.index = retornos.index.to_period('M').to_timestamp()        return retornosdef converter_retorno_usd_para_brl(r_usd: pd.Series, r_fx: pd.Series) -> pd.Series:    """    Converte retornos de USD para BRL usando a fórmula:    r_BRL = (1 + r_USD) * (1 + r_FX) - 1        Onde r_FX é o retorno do câmbio USD/BRL.    """    # Alinhar índices    idx_comum = r_usd.index.intersection(r_fx.index)        r_usd_alinhado = r_usd.loc[idx_comum]    r_fx_alinhado = r_fx.loc[idx_comum]        # Aplicar fórmula    r_brl = (1 + r_usd_alinhado) * (1 + r_fx_alinhado) - 1        return r_brlprint("✅ Funções de yfinance e câmbio carregadas!")

In [ ]:
# =============================================================================# FUNÇÕES - BUSCA E SELEÇÃO DE ARQUIVO EXCEL# =============================================================================def varrer_arquivos_excel(diretorio: str, pattern: str = "*.xlsx", recursive: bool = False) -> List[Dict]:    """Varre diretório buscando arquivos Excel."""    dir_path = Path(diretorio)        if not dir_path.exists():        alternativas = [diretorio.replace("/", "\\"), diretorio.replace("\\", "/")]        for alt in alternativas:            if Path(alt).exists():                dir_path = Path(alt)                break        else:            return []        arquivos = list(dir_path.rglob(pattern) if recursive else dir_path.glob(pattern))        # Case insensitive    if pattern.endswith(".xlsx"):        arquivos.extend(dir_path.glob(pattern.replace(".xlsx", ".XLSX")))        resultado = []    for arq in arquivos:        nome = arq.name        if nome.startswith("~$") or nome.startswith("."):            continue                try:            stat = arq.stat()            resultado.append({                'path': arq, 'nome': nome,                'modificado': datetime.fromtimestamp(stat.st_mtime),                'modificado_str': datetime.fromtimestamp(stat.st_mtime).strftime("%Y-%m-%d %H:%M"),                'tamanho_kb': stat.st_size / 1024            })        except:            pass        resultado.sort(key=lambda x: x['modificado'], reverse=True)    return resultadodef selecionar_arquivo_excel_interativo(diretorio: str, pattern: str = "*.xlsx",                                        arquivo_padrao: str = None) -> str:    """Seleciona arquivo Excel com interação."""    print("=" * 60)    print("SELEÇÃO DE ARQUIVO EXCEL")    print("=" * 60)        arquivos = varrer_arquivos_excel(diretorio, pattern)        if not arquivos:        print(f"\n❌ Nenhum arquivo Excel em: {diretorio}")        raise FileNotFoundError(f"Nenhum arquivo Excel em {diretorio}")        print(f"\n📂 Encontrados {len(arquivos)} arquivo(s):")        idx_padrao = None    for i, arq in enumerate(arquivos):        if arquivo_padrao and arq['nome'] == arquivo_padrao:            idx_padrao = i        marcador = " ⭐" if i == idx_padrao else ""        print(f"   [{i}] {arq['nome']}{marcador} ({arq['modificado_str']})")        if idx_padrao is not None and len(arquivos) > 1:        try:            usar = input(f"\n   Usar '{arquivo_padrao}'? (s/n) [s]: ").strip().lower()            if usar == "" or usar == "s":                return str(arquivos[idx_padrao]['path'])        except:            return str(arquivos[idx_padrao]['path'])        if len(arquivos) == 1:        print(f"\n✅ Selecionado: {arquivos[0]['nome']}")        return str(arquivos[0]['path'])        try:        escolha = input("   Escolha [ENTER=mais recente]: ").strip()        idx = 0 if escolha == "" else int(escolha)    except:        idx = 0        idx = max(0, min(idx, len(arquivos)-1))    print(f"\n✅ Selecionado: {arquivos[idx]['nome']}")    return str(arquivos[idx]['path'])print("✅ Funções de arquivo carregadas!")

In [ ]:
# =============================================================================# FUNÇÕES - LEITURA E PROCESSAMENTO# =============================================================================def normalizar_nome(nome: str) -> str:    if pd.isna(nome):        return ""    nome = str(nome).lower().strip()    acentos = {'á':'a','à':'a','ã':'a','â':'a','é':'e','ê':'e','í':'i',               'ó':'o','ô':'o','õ':'o','ú':'u','ç':'c'}    for k, v in acentos.items():        nome = nome.replace(k, v)    nome = re.sub(r'[-/_]', ' ', nome)    nome = re.sub(r'[^a-z0-9\s]', ' ', nome)    return re.sub(r'\s+', ' ', nome).strip()def similaridade(a: str, b: str) -> float:    na, nb = normalizar_nome(a), normalizar_nome(b)    score = SequenceMatcher(None, na, nb).ratio()    if na in nb or nb in na:        score = max(score, 0.85)    words_a, words_b = set(na.split()), set(nb.split())    if len(words_a & words_b) >= 2:        score = max(score, 0.75 + 0.05 * len(words_a & words_b))    return min(score, 1.0)def parse_percentual(valor) -> float:    if pd.isna(valor) or valor in ['-', '']:        return np.nan    if isinstance(valor, (int, float)):        return valor / 100 if abs(valor) > 1 else valor    try:        return float(str(valor).replace('%', '').replace(',', '.')) / 100    except:        return np.nandef ler_retornos_aba_excel(xlsx_path: str, sheet_name: str) -> pd.Series:    df = pd.read_excel(xlsx_path, sheet_name=sheet_name, header=None, engine='openpyxl')        header_row = None    for i, row in df.iterrows():        if str(row.iloc[0]).strip().lower() == 'ano':            header_row = i            break        if header_row is None:        raise ValueError(f"Cabeçalho 'Ano' não encontrado em '{sheet_name}'")        meses = {'jan':1,'fev':2,'mar':3,'abr':4,'mai':5,'jun':6,             'jul':7,'ago':8,'set':9,'out':10,'nov':11,'dez':12}    header = [str(c).strip().lower()[:3] for c in df.iloc[header_row].values]        retornos = []    for i in range(header_row + 1, len(df)):        row = df.iloc[i]        try:            ano = int(row.iloc[0])        except:            break                for col_idx, col_name in enumerate(header[1:], start=1):            if col_name in meses:                val = parse_percentual(row.iloc[col_idx])                if not pd.isna(val):                    retornos.append({'data': pd.Timestamp(ano, meses[col_name], 1), 'retorno': val})        if not retornos:        return pd.Series(dtype=float)        return pd.DataFrame(retornos).set_index('data').sort_index()['retorno']def ler_pesos_resumo_robusto(xlsx_path: str) -> Tuple[Dict[str, float], str]:    try:        df = pd.read_excel(xlsx_path, sheet_name='Resumo', header=None, engine='openpyxl')    except:        return {}, "erro"        col_ativo = col_peso = header_row = None        for i, row in df.iterrows():        for j, val in enumerate(row):            if pd.isna(val):                continue            val_str = str(val).strip().lower()            if val_str in ['ativo', 'asset']:                col_ativo = j            if any(kw in val_str for kw in ['peso', 'weight']) and any(kw in val_str for kw in ['cart', 'atual']):                col_peso = j        if col_ativo is not None and col_peso is not None:            header_row = i            break        if col_peso is None:        return {}, "nao_encontrado"        pesos = {}    for i in range(header_row + 1, len(df)):        ativo = df.iloc[i, col_ativo]        peso = df.iloc[i, col_peso]        if pd.isna(ativo) or str(ativo).strip() == '':            continue        try:            peso_val = float(peso) if not pd.isna(peso) else 0.0            if peso_val > 1:                peso_val /= 100            pesos[str(ativo).strip()] = peso_val        except:            pesos[str(ativo).strip()] = 0.0        return pesos, "encontrado"print("✅ Funções de leitura carregadas!")

In [ ]:
# =============================================================================# ESTRUTURA DE METADADOS POR ATIVO# =============================================================================class MetadadosAtivo:    """Metadados de um ativo."""    def __init__(self, nome: str, moeda: str = "BRL", fonte: str = "excel", ticker: str = None):        self.nome = nome        self.moeda = moeda.upper()  # BRL ou USD        self.fonte = fonte.lower()  # excel, yfinance, manual        self.ticker = ticker        self.convertido = False  # Se foi convertido USD→BRL        def to_dict(self):        return {            'nome': self.nome, 'moeda': self.moeda, 'fonte': self.fonte,            'ticker': self.ticker, 'convertido': self.convertido        }def criar_metadados_ativos(lista_ativos: List[str], ativos_usd: List[str]) -> Dict[str, MetadadosAtivo]:    """Cria dicionário de metadados identificando moeda."""    metadados = {}    for ativo in lista_ativos:        # Verificar se é USD        moeda = "USD" if any(usd.upper() in ativo.upper() for usd in ativos_usd) else "BRL"        metadados[ativo] = MetadadosAtivo(nome=ativo, moeda=moeda, fonte="excel")    return metadadosdef exibir_resumo_metadados(metadados: Dict[str, MetadadosAtivo]):    """Exibe resumo dos metadados."""    usd = [m.nome for m in metadados.values() if m.moeda == "USD"]    brl = [m.nome for m in metadados.values() if m.moeda == "BRL"]    convertidos = [m.nome for m in metadados.values() if m.convertido]        print(f"\n📊 RESUMO DE METADADOS:")    print(f"   Total: {len(metadados)} ativos")    print(f"   BRL: {len(brl)} | USD: {len(usd)}")    if usd:        print(f"   Ativos USD: {', '.join(usd[:10])}{'...' if len(usd) > 10 else ''}")    if convertidos:        print(f"   Convertidos USD→BRL: {len(convertidos)}")print("✅ Estrutura de metadados carregada!")

In [ ]:
# =============================================================================# FUNÇÕES - CONVERSÃO DE MOEDA# =============================================================================def obter_retornos_cambio(data_inicio: str, data_fim: str,                           fx_ticker: str = "USDBRL=X",                          cache_dir: str = None) -> pd.Series:    """    Obtém retornos mensais do câmbio USD/BRL.    """    if not YFINANCE_DISPONIVEL:        print("   ⚠️ yfinance não disponível, usando câmbio fixo")        return pd.Series(dtype=float)        print(f"\n📥 Obtendo série de câmbio {fx_ticker}...")    precos_fx = baixar_cambio_usdbrl(data_inicio, data_fim, fx_ticker, cache_dir)        if precos_fx.empty:        print("   ❌ Falha ao obter câmbio")        return pd.Series(dtype=float)        retornos_fx = calcular_retornos_mensais(precos_fx)    print(f"   ✅ {len(retornos_fx)} meses de câmbio obtidos")    print(f"   Período: {retornos_fx.index.min().strftime('%Y-%m')} a {retornos_fx.index.max().strftime('%Y-%m')}")        return retornos_fxdef converter_ativos_usd_para_brl(retornos_df: pd.DataFrame,                                   metadados: Dict[str, MetadadosAtivo],                                  retornos_fx: pd.Series) -> pd.DataFrame:    """    Converte retornos de ativos USD para BRL.    Modifica metadados in-place para marcar convertidos.    """    if retornos_fx.empty:        print("   ⚠️ Sem série de câmbio, mantendo retornos originais")        return retornos_df        retornos_convertidos = retornos_df.copy()    n_convertidos = 0        for ativo in retornos_df.columns:        if ativo in metadados and metadados[ativo].moeda == "USD":            r_usd = retornos_df[ativo]            r_brl = converter_retorno_usd_para_brl(r_usd, retornos_fx)                        if not r_brl.empty:                # Preencher gaps com valores originais                retornos_convertidos[ativo] = r_brl.reindex(retornos_df.index)                retornos_convertidos[ativo] = retornos_convertidos[ativo].fillna(r_usd)                metadados[ativo].convertido = True                n_convertidos += 1        print(f"   ✅ {n_convertidos} ativos convertidos USD→BRL")    return retornos_convertidosdef validar_conversao_fx(ativo: str, retornos_usd: pd.Series,                          retornos_fx: pd.Series, retornos_brl: pd.Series, n_linhas: int = 5):    """    Exibe validação da conversão FX para um ativo.    """    print(f"\n🔍 VALIDAÇÃO DE CONVERSÃO: {ativo}")    print("-" * 70)    print(f"{'Data':<12} {'r_USD':>10} {'r_FX':>10} {'r_BRL calc':>12} {'r_BRL real':>12} {'OK?':>6}")    print("-" * 70)        idx_comum = retornos_usd.index.intersection(retornos_fx.index).intersection(retornos_brl.index)[:n_linhas]        for data in idx_comum:        r_usd = retornos_usd.loc[data]        r_fx = retornos_fx.loc[data]        r_brl_calc = (1 + r_usd) * (1 + r_fx) - 1        r_brl_real = retornos_brl.loc[data]        ok = "✅" if abs(r_brl_calc - r_brl_real) < 0.0001 else "❌"        print(f"{data.strftime('%Y-%m'):<12} {r_usd:>10.4f} {r_fx:>10.4f} {r_brl_calc:>12.4f} {r_brl_real:>12.4f} {ok:>6}")print("✅ Funções de conversão FX carregadas!")

In [ ]:
# =============================================================================# FUNÇÕES - MAPEAMENTO DE PESOS# =============================================================================def mapear_pesos_para_abas(pesos_resumo: Dict[str, float], abas_candidatas: List[str],                           abas_excluidas_min_obs: List[str], ativos_excluir: List[str],                           threshold_ok: float = 0.90, threshold_revisar: float = 0.75) -> pd.DataFrame:    """Mapeia ativos do Resumo para abas."""    resultados = []    todas_abas = set(abas_candidatas) | set(abas_excluidas_min_obs)        for ativo, peso in pesos_resumo.items():        if peso == 0:            continue                # Verificar exclusão manual        if any(excl.upper() in ativo.upper() for excl in ativos_excluir):            resultados.append({                'ativo_resumo': ativo, 'peso': peso, 'aba_mapeada': None,                'score': 0, 'status': 'EXCLUIDO_MANUAL', 'motivo': f'Excluído: {ativos_excluir}'            })            continue                # Match exato        if ativo in todas_abas:            status = "MAPEADO_EXCLUIDO_MIN_OBS" if ativo in abas_excluidas_min_obs else "OK"            resultados.append({                'ativo_resumo': ativo, 'peso': peso, 'aba_mapeada': ativo,                'score': 1.0, 'status': status, 'motivo': 'Match exato'            })            continue                # Fuzzy match        melhor_match, melhor_score = None, 0        for aba in todas_abas:            score = similaridade(ativo, aba)            if score > melhor_score:                melhor_score, melhor_match = score, aba                if melhor_score >= threshold_ok:            status = "MAPEADO_EXCLUIDO_MIN_OBS" if melhor_match in abas_excluidas_min_obs else "OK"        elif melhor_score >= threshold_revisar:            status = "MAPEADO_EXCLUIDO_MIN_OBS" if melhor_match in abas_excluidas_min_obs else "REVISAR"        else:            status, melhor_match = "NAO_MAPEADO", None                resultados.append({            'ativo_resumo': ativo, 'peso': peso, 'aba_mapeada': melhor_match,            'score': melhor_score, 'status': status, 'motivo': f'Score: {melhor_score:.2f}'        })        return pd.DataFrame(resultados)def calcular_cobertura_pesos(df_mapa: pd.DataFrame) -> Dict:    """Calcula métricas de cobertura de pesos."""    peso_total = df_mapa['peso'].sum()        return {        'peso_total_resumo': peso_total,        'peso_ok': df_mapa[df_mapa['status'] == 'OK']['peso'].sum(),        'peso_revisar': df_mapa[df_mapa['status'] == 'REVISAR']['peso'].sum(),        'peso_excluido_min_obs': df_mapa[df_mapa['status'] == 'MAPEADO_EXCLUIDO_MIN_OBS']['peso'].sum(),        'peso_excluido_manual': df_mapa[df_mapa['status'] == 'EXCLUIDO_MANUAL']['peso'].sum(),        'peso_nao_mapeado': df_mapa[df_mapa['status'] == 'NAO_MAPEADO']['peso'].sum(),        'n_ok': (df_mapa['status'] == 'OK').sum(),        'n_revisar': (df_mapa['status'] == 'REVISAR').sum(),        'n_excluido_min_obs': (df_mapa['status'] == 'MAPEADO_EXCLUIDO_MIN_OBS').sum(),        'n_excluido_manual': (df_mapa['status'] == 'EXCLUIDO_MANUAL').sum(),        'n_nao_mapeado': (df_mapa['status'] == 'NAO_MAPEADO').sum(),    }def exibir_auditoria_cobertura(cobertura: Dict, politica: str):    """Exibe auditoria detalhada de cobertura."""    print("\n" + "=" * 60)    print("🔍 AUDITORIA DE COBERTURA DE PESOS")    print("=" * 60)        peso_total = cobertura['peso_total_resumo']    peso_no_universo = cobertura['peso_ok'] + cobertura['peso_revisar']    peso_fora = peso_total - peso_no_universo        print(f"\n   Peso total no Resumo:           {peso_total:.2%}")    print(f"   ├─ OK no universo:               {cobertura['peso_ok']:.2%} ({cobertura['n_ok']} ativos)")    print(f"   ├─ REVISAR no universo:          {cobertura['peso_revisar']:.2%} ({cobertura['n_revisar']} ativos)")    print(f"   ├─ Excluído por MIN_OBS:         {cobertura['peso_excluido_min_obs']:.2%} ({cobertura['n_excluido_min_obs']} ativos)")    print(f"   ├─ Excluído manualmente:         {cobertura['peso_excluido_manual']:.2%} ({cobertura['n_excluido_manual']} ativos)")    print(f"   └─ Não mapeado:                  {cobertura['peso_nao_mapeado']:.2%} ({cobertura['n_nao_mapeado']} ativos)")        print(f"\n   TOTAL NO UNIVERSO:              {peso_no_universo:.2%}")    print(f"   TOTAL FORA DO UNIVERSO:          {peso_fora:.2%}")        print(f"\n   Política configurada:           {politica.upper()}")        if peso_fora > 0.01:        if politica == "reportar":            print(f"   ⚠️ {peso_fora:.1%} do peso ficará FORA da otimização")        elif politica == "renormalizar":            print(f"   ⚠️ Pesos serão renormalizados (base: {peso_no_universo:.2%})")        elif politica == "caixa_cdi":            print(f"   ⚠️ {peso_fora:.1%} será tratado como Caixa CDI")print("✅ Funções de mapeamento carregadas!")

In [ ]:
# =============================================================================# FUNÇÕES - CARREGAMENTO DE DADOS# =============================================================================def carregar_metadados_excel(xlsx_path: str, ativos_excluir: List[str] = None):    if ativos_excluir is None:        ativos_excluir = []        xlsx = pd.ExcelFile(xlsx_path, engine='openpyxl')        metadados = {        'abas_candidatas': [],        'meses_por_ativo': {},        'ativos_excluidos_manual': [],        'erros': []    }        for aba in xlsx.sheet_names:        if aba.lower() == 'resumo':            continue        if any(excl.upper() in aba.upper() for excl in ativos_excluir):            metadados['ativos_excluidos_manual'].append(aba)            continue                try:            serie = ler_retornos_aba_excel(xlsx_path, aba)            metadados['meses_por_ativo'][aba] = len(serie.dropna())            metadados['abas_candidatas'].append(aba)        except Exception as e:            metadados['erros'].append((aba, str(e)[:50]))        return metadadosdef filtrar_por_min_obs(metadados: Dict, min_obs: int) -> Tuple[List[str], List[str]]:    abas_ok, abas_excluidas = [], []    for aba, n in metadados['meses_por_ativo'].items():        (abas_ok if n >= min_obs else abas_excluidas).append(aba)    return abas_ok, abas_excluidasdef carregar_retornos_das_abas(xlsx_path: str, abas: List[str]) -> pd.DataFrame:    series = {}    for aba in abas:        series[aba] = ler_retornos_aba_excel(xlsx_path, aba)    return pd.DataFrame(series)print("✅ Funções de carregamento carregadas!")

In [ ]:
# =============================================================================# FUNÇÕES - ESTATÍSTICAS# =============================================================================def retorno_geometrico_anual(ret: pd.Series) -> float:    ret = ret.dropna()    if len(ret) == 0:        return np.nan    return float(np.prod(1 + ret) ** (12 / len(ret)) - 1)def retorno_aritmetico_anual(ret: pd.Series) -> float:    return float(ret.mean() * 12)def calcular_matriz_overlap(df: pd.DataFrame) -> pd.DataFrame:    n = len(df.columns)    overlap = pd.DataFrame(index=df.columns, columns=df.columns, dtype=float)    for i, a in enumerate(df.columns):        for j, b in enumerate(df.columns):            overlap.iloc[i, j] = (df[a].notna() & df[b].notna()).sum() if i != j else len(df[a].dropna())    return overlapdef aplicar_min_overlap(cov: pd.DataFrame, overlap: pd.DataFrame, min_overlap: int) -> Tuple[pd.DataFrame, int]:    cov_adj = cov.copy()    n_zerados = 0    for i, a in enumerate(cov.index):        for j, b in enumerate(cov.columns):            if i != j and overlap.iloc[i, j] < min_overlap:                cov_adj.iloc[i, j] = cov_adj.iloc[j, i] = 0                n_zerados += 1    return cov_adj, n_zerados // 2def aplicar_vol_floor(cov: pd.DataFrame, floor: float):    cov_adj = cov.copy()    floor_anual = (floor ** 2) * 12    ajustados = []    for i, a in enumerate(cov.index):        if cov.iloc[i, i] < floor_anual:            cov_adj.iloc[i, i] = floor_anual            ajustados.append(a)    return cov_adj, ajustadosdef corrigir_psd(cov: pd.DataFrame, eps: float = 1e-8):    vals, vecs = eigh(cov.values)    if np.min(vals) >= -eps:        return cov, False    vals_corr = np.maximum(vals, eps)    arr = vecs @ np.diag(vals_corr) @ vecs.T    return pd.DataFrame((arr + arr.T) / 2, index=cov.index, columns=cov.columns), Trueprint("✅ Funções de estatística carregadas!")

In [ ]:
# =============================================================================# FUNÇÕES - OTIMIZAÇÃO# =============================================================================class ResultadoOtimizacao:    def __init__(self, pesos, sucesso, msg, metodo):        self.pesos = pesos        self.sucesso = sucesso        self.mensagem = msg        self.metodo = metododef calcular_metricas(pesos, ret_esp, cov, rf=0):    pesos = np.array(pesos)    retorno = float(np.dot(pesos, ret_esp))    vol = float(np.sqrt(np.dot(pesos.T, np.dot(cov, pesos))))    sharpe = (retorno - rf) / vol if vol > 1e-8 else np.nan    return retorno, vol, sharpedef otimizar_gmv(ret_esp, cov, peso_min=0, peso_max=1):    n = len(ret_esp)    x0 = np.ones(n) / n    bounds = [(peso_min, peso_max)] * n    cons = {'type': 'eq', 'fun': lambda x: np.sum(x) - 1}        res = minimize(lambda w: np.sqrt(np.dot(w.T, np.dot(cov, w))), x0,                    method='SLSQP', bounds=bounds, constraints=cons, options={'maxiter': 1000})        if res.success:        return ResultadoOtimizacao(res.x, True, "OK", "GMV")    return ResultadoOtimizacao(np.ones(n)/n, False, res.message, "GMV_FALLBACK")def otimizar_max_sharpe(ret_esp, cov, rf=0, peso_min=0, peso_max=1):    n = len(ret_esp)    x0 = np.ones(n) / n    bounds = [(peso_min, peso_max)] * n    cons = {'type': 'eq', 'fun': lambda x: np.sum(x) - 1}        def neg_sharpe(w):        ret = np.dot(w, ret_esp)        vol = np.sqrt(np.dot(w.T, np.dot(cov, w)))        return -(ret - rf) / vol if vol > 1e-8 else 1e10        res = minimize(neg_sharpe, x0, method='SLSQP', bounds=bounds, constraints=cons, options={'maxiter': 1000})        if res.success:        return ResultadoOtimizacao(res.x, True, "OK", "MAX_SHARPE")    return ResultadoOtimizacao(np.ones(n)/n, False, res.message, "MAX_SHARPE_FALLBACK")def calcular_fronteira(ret_esp, cov, rf, n_pontos=50, peso_min=0, peso_max=1):    gmv = otimizar_gmv(ret_esp, cov, peso_min, peso_max)    ret_gmv, vol_gmv, _ = calcular_metricas(gmv.pesos, ret_esp, cov, rf)        ret_max = np.max(ret_esp)    targets = np.linspace(ret_gmv, ret_max * 0.95, n_pontos)        vols, rets = [], []    for target in targets:        n = len(ret_esp)        x0 = np.ones(n) / n        bounds = [(peso_min, peso_max)] * n        cons = [{'type': 'eq', 'fun': lambda x: np.sum(x) - 1},                {'type': 'eq', 'fun': lambda x, t=target: np.dot(x, ret_esp) - t}]                res = minimize(lambda w: np.dot(w.T, np.dot(cov, w)), x0, method='SLSQP',                        bounds=bounds, constraints=cons)        if res.success:            r, v, _ = calcular_metricas(res.x, ret_esp, cov, rf)            vols.append(v)            rets.append(r)        return np.array(vols), np.array(rets)print("✅ Funções de otimização carregadas!")

In [ ]:
# =============================================================================# FUNÇÕES - MONTE CARLO# =============================================================================def simular_monte_carlo(ret_esp, cov, rf, n_port=50000, seed=42,                        alpha_conc=0.2, alpha_div=1.0, frac_cantos=0.05, frac_sparse=0.15,                        peso_min=0, peso_max=1):    """Monte Carlo com respeito aos bounds."""    np.random.seed(seed)    n = len(ret_esp)        n_cantos = int(n_port * frac_cantos)    n_sparse = int(n_port * frac_sparse)    n_conc = int((n_port - n_cantos - n_sparse) * 0.6)    n_div = n_port - n_cantos - n_sparse - n_conc        pesos = []        # Cantos    for _ in range(n_cantos):        w = np.zeros(n)        idx = np.random.randint(n)        w[idx] = min(peso_max, np.random.uniform(0.85, 0.98))        resto = 1 - w[idx]        outros = [i for i in range(n) if i != idx]        if outros and resto > 0:            w[outros] = np.random.dirichlet([0.5] * len(outros)) * resto        w = np.clip(w, peso_min, peso_max)        w /= w.sum()        pesos.append(w)        # Sparse    for _ in range(n_sparse):        w = np.zeros(n)        k = np.random.randint(3, min(9, n + 1))        idx = np.random.choice(n, k, replace=False)        w[idx] = np.random.dirichlet([0.5] * k)        w = np.clip(w, peso_min, peso_max)        w /= w.sum()        pesos.append(w)        # Concentrado    for _ in range(n_conc):        w = np.random.dirichlet([alpha_conc] * n)        w = np.clip(w, peso_min, peso_max)        w /= w.sum()        pesos.append(w)        # Diversificado    for _ in range(n_div):        w = np.random.dirichlet([alpha_div] * n)        w = np.clip(w, peso_min, peso_max)        w /= w.sum()        pesos.append(w)        pesos = np.array(pesos)    rets = pesos @ ret_esp    vols = np.sqrt(np.einsum('ij,jk,ik->i', pesos, cov, pesos))    sharpes = np.where(vols > 1e-8, (rets - rf) / vols, np.nan)        return rets, vols, sharpes, pesosprint("✅ Funções Monte Carlo carregadas!")

In [ ]:
# =============================================================================# FUNÇÕES - CDI# =============================================================================def obter_cdi_atual():    try:        url = "https://api.bcb.gov.br/dados/serie/bcdata.sgs.12/dados/ultimos/1?formato=json"        df = pd.read_json(url)        if df.empty:            return None        taxa = float(str(df.loc[0, "valor"]).replace(",", ".")) / 100        return float((1 + taxa) ** 252 - 1)    except:        return Nonedef obter_cdi_periodo(di, df):    try:        di_str = pd.to_datetime(di).strftime("%d/%m/%Y")        df_str = pd.to_datetime(df).strftime("%d/%m/%Y")        url = f"https://api.bcb.gov.br/dados/serie/bcdata.sgs.12/dados?formato=json&dataInicial={di_str}&dataFinal={df_str}"        cdi = pd.read_json(url)        if cdi.empty:            return None        cdi["valor"] = pd.to_numeric(cdi["valor"].astype(str).str.replace(",", "."), errors="coerce") / 100        return float(np.expm1(np.log1p(cdi["valor"].dropna()).mean() * 252))    except:        return Nonedef obter_rf(modo, fallback, manual=None, di=None, df=None):    if modo == "manual":        return manual or fallback    if modo == "atual":        return obter_cdi_atual() or fallback    if modo == "media_periodo" and di and df:        return obter_cdi_periodo(di, df) or obter_cdi_atual() or fallback    return fallbackprint("✅ Funções CDI carregadas!")

In [ ]:
# =============================================================================# FUNÇÕES - VISUALIZAÇÃO# =============================================================================def criar_grafico_fronteira(ret_sim, vol_sim, sharpe_sim, front_vols, front_rets,                            carteiras, rf, titulo="Fronteira Eficiente"):    fig = go.Figure()        fig.add_trace(go.Scatter(        x=vol_sim*100, y=ret_sim*100, mode='markers',        marker=dict(size=2, color=sharpe_sim, colorscale='Viridis',                     colorbar=dict(title='Sharpe'), opacity=0.4),        name='Simulados'    ))        fig.add_trace(go.Scatter(        x=front_vols*100, y=front_rets*100, mode='lines',        line=dict(color='red', width=3), name='Fronteira'    ))        estilos = {        'GMV': {'color': 'blue', 'symbol': 'diamond', 'size': 18},        'Max Sharpe': {'color': 'green', 'symbol': 'triangle-up', 'size': 18},        'Carteira Atual': {'color': 'gold', 'symbol': 'star', 'size': 24}    }        for nome, dados in carteiras.items():        e = estilos.get(nome, {'color': 'purple', 'symbol': 'circle', 'size': 15})        fig.add_trace(go.Scatter(            x=[dados['vol']*100], y=[dados['ret']*100], mode='markers+text',            marker=dict(size=e['size'], color=e['color'], symbol=e['symbol'],                        line=dict(width=2, color='black')),            text=[nome], textposition='top center', name=nome        ))        fig.add_hline(y=rf*100, line_dash="dash", line_color="gray",                   annotation_text=f"RF: {rf*100:.1f}%")        fig.update_layout(title=titulo, xaxis_title="Volatilidade (%)", yaxis_title="Retorno (%)",                      template="plotly_white", width=1100, height=700)    return figdef criar_heatmap(corr, titulo="Correlação"):    fig = go.Figure(data=go.Heatmap(z=corr.values, x=list(corr.columns), y=list(corr.index),                                     colorscale='RdBu', zmid=0))    fig.update_layout(title=titulo, width=900, height=800)    return figdef criar_barras_pesos(carteiras, ativos, top_n=15):    fig = go.Figure()    cores = {'GMV': '#636EFA', 'Max Sharpe': '#00CC96', 'Carteira Atual': '#FFA15A'}        for nome, dados in carteiras.items():        pesos = dados.get('pesos', [])        if len(pesos) == 0:            continue        df = pd.DataFrame({'ativo': ativos, 'peso': pesos}).nlargest(top_n, 'peso')        fig.add_trace(go.Bar(name=nome, x=df['ativo'], y=df['peso']*100,                             marker_color=cores.get(nome, '#AB63FA')))        fig.update_layout(title="Comparativo de Pesos", barmode='group', template="plotly_white")    return figprint("✅ Funções de visualização carregadas!")

---## 📥 BLOCO 2 - CARREGAMENTO DE DADOS

In [ ]:
# =============================================================================# ETAPA 1: PERGUNTAS INICIAIS# =============================================================================print("=" * 60)print("CONFIGURAÇÃO INICIAL")print("=" * 60)# Perguntar sobre Exceltry:    usar_excel = input("\nCarregar dados do Excel? (s/n) [s]: ").strip().lower()    USAR_EXCEL = usar_excel == "" or usar_excel == "s"except:    USAR_EXCEL = True# Perguntar sobre yfinance fallbackif USAR_YFINANCE and YFINANCE_DISPONIVEL:    try:        usar_yf = input("Se faltar ativo, tentar yfinance? (s/n) [s]: ").strip().lower()        USAR_YFINANCE_FALLBACK = usar_yf == "" or usar_yf == "s"    except:        USAR_YFINANCE_FALLBACK = Trueelse:    USAR_YFINANCE_FALLBACK = Falseprint(f"\n   Excel: {'SIM' if USAR_EXCEL else 'NÃO'}")print(f"   yfinance fallback: {'SIM' if USAR_YFINANCE_FALLBACK else 'NÃO'}")

In [ ]:
# =============================================================================# ETAPA 2: CARREGAR DADOS# =============================================================================retornos_df = Nonemetadados_ativos = {}EXCEL_PATH = Noneif USAR_EXCEL:    try:        EXCEL_PATH = selecionar_arquivo_excel_interativo(EXCEL_DIR, EXCEL_PATTERN, ARQUIVO_PADRAO)                print("\n📂 Carregando metadados...")        meta_excel = carregar_metadados_excel(EXCEL_PATH, ATIVOS_EXCLUIR)                print(f"   Abas candidatas: {len(meta_excel['abas_candidatas'])}")        print(f"   Excluídos manual: {meta_excel['ativos_excluidos_manual']}")                abas_ok, abas_excluidas_min_obs = filtrar_por_min_obs(meta_excel, MIN_OBS)        print(f"   Passaram MIN_OBS: {len(abas_ok)}")                retornos_df = carregar_retornos_das_abas(EXCEL_PATH, abas_ok)        metadados_ativos = criar_metadados_ativos(list(retornos_df.columns), ATIVOS_USD)                print(f"\n✅ Excel carregado: {len(retornos_df.columns)} ativos")            except Exception as e:        print(f"\n❌ Erro ao carregar Excel: {e}")        USAR_EXCEL = Falseif not USAR_EXCEL and USAR_YFINANCE_FALLBACK:    print("\n📥 Usando yfinance com DEFAULT_UNIVERSE...")    # Implementação simplificada - baixar do yfinance    # (expandir conforme necessário)

In [ ]:
# =============================================================================# ETAPA 3: OBTER SÉRIE DE CÂMBIO E CONVERTER USD→BRL# =============================================================================retornos_fx = pd.Series(dtype=float)ativos_usd_encontrados = [a for a in retornos_df.columns if metadados_ativos.get(a, MetadadosAtivo(a)).moeda == "USD"]if ativos_usd_encontrados and YFINANCE_DISPONIVEL:    print(f"\n📊 Encontrados {len(ativos_usd_encontrados)} ativos em USD: {ativos_usd_encontrados[:5]}...")        # Obter período dos dados    data_inicio = retornos_df.index.min().strftime('%Y-%m-%d')    data_fim = retornos_df.index.max().strftime('%Y-%m-%d')        # Baixar câmbio    retornos_fx = obter_retornos_cambio(data_inicio, data_fim, FX_TICKER, CACHE_DIR if USAR_YFINANCE else None)        if not retornos_fx.empty:        # Guardar retornos originais para validação        retornos_usd_original = retornos_df[ativos_usd_encontrados].copy()                # Converter        print("\n🔄 Convertendo retornos USD → BRL...")        retornos_df = converter_ativos_usd_para_brl(retornos_df, metadados_ativos, retornos_fx)                # Validar primeiro ativo USD        if ativos_usd_encontrados:            ativo_validacao = ativos_usd_encontrados[0]            validar_conversao_fx(                ativo_validacao,                retornos_usd_original[ativo_validacao],                retornos_fx,                retornos_df[ativo_validacao]            )else:    print("\n📊 Nenhum ativo USD encontrado ou yfinance indisponível")# Exibir resumoexibir_resumo_metadados(metadados_ativos)

In [ ]:
# =============================================================================# ETAPA 4: CARREGAR PESOS (OPCIONAL)# =============================================================================print("\n" + "=" * 60)print("CARTEIRA ATUAL")print("=" * 60)TEM_PESOS = Falsedf_mapa = Nonecobertura = Noneif EXCEL_PATH:    pesos_resumo, status = ler_pesos_resumo_robusto(EXCEL_PATH)        if status == "encontrado":        n_com_peso = sum(1 for v in pesos_resumo.values() if v > 0)        print(f"\n   Aba Resumo: {n_com_peso} ativos com peso > 0")                try:            carregar = input("   Carregar pesos da carteira atual? (s/n) [s]: ").strip().lower()            CARREGAR_PESOS = carregar == "" or carregar == "s"        except:            CARREGAR_PESOS = True                if CARREGAR_PESOS:            df_mapa = mapear_pesos_para_abas(                pesos_resumo, abas_ok, abas_excluidas_min_obs, ATIVOS_EXCLUIR,                THRESHOLD_OK, THRESHOLD_REVISAR            )                        cobertura = calcular_cobertura_pesos(df_mapa)            exibir_auditoria_cobertura(cobertura, POLITICA_PESO_FORA)                        print("\n📋 Mapeamento:")            display(df_mapa.sort_values('peso', ascending=False).head(20))                        TEM_PESOS = True    else:        print("\n   ⚠️ Coluna de pesos não encontrada")

In [ ]:
# =============================================================================# ETAPA 5: CRIAR VETOR DE PESOS# =============================================================================ATIVOS_OTIMIZACAO = list(retornos_df.columns)NUM_ATIVOS = len(ATIVOS_OTIMIZACAO)pesos_carteira = NoneTEM_CARTEIRA_ATUAL = Falseif TEM_PESOS and df_mapa is not None:    df_ok = df_mapa[df_mapa['status'].isin(['OK', 'REVISAR'])]        pesos_carteira = np.zeros(NUM_ATIVOS)    for _, row in df_ok.iterrows():        aba = row['aba_mapeada']        if aba in ATIVOS_OTIMIZACAO:            pesos_carteira[ATIVOS_OTIMIZACAO.index(aba)] += row['peso']        soma = pesos_carteira.sum()        if soma > 0:        if POLITICA_PESO_FORA == "renormalizar" and abs(soma - 1) > 0.01:            print(f"\n   Renormalizando: {soma:.2%} → 100%")            pesos_carteira /= soma        TEM_CARTEIRA_ATUAL = True        print(f"\n✅ Carteira atual: {soma:.2%} do peso no universo")print(f"\n📊 Universo de otimização: {NUM_ATIVOS} ativos")

---## 📈 BLOCO 3 - CÁLCULO DE ESTATÍSTICAS

In [ ]:
# =============================================================================# CDI E ESTATÍSTICAS# =============================================================================print("\n" + "=" * 60)print("ESTATÍSTICAS")print("=" * 60)periodo_inicio = retornos_df.index.min().strftime('%Y-%m-%d')periodo_fim = retornos_df.index.max().strftime('%Y-%m-%d')RF_ANUAL = obter_rf(RF_MODO, RF_FALLBACK, RF_MANUAL, periodo_inicio, periodo_fim)print(f"\n📊 RF (CDI): {RF_ANUAL*100:.2f}% a.a.")# Retornos esperadosif USAR_MEDIA_GEOMETRICA:    ret_esperados = retornos_df.apply(retorno_geometrico_anual)else:    ret_esperados = retornos_df.apply(retorno_aritmetico_anual)# Covariânciacov_matrix = retornos_df.cov() * 12# MIN_OVERLAPoverlap = calcular_matriz_overlap(retornos_df)cov_matrix, n_zerados = aplicar_min_overlap(cov_matrix, overlap, MIN_OVERLAP)if n_zerados > 0:    print(f"   ⚠️ MIN_OVERLAP: {n_zerados} pares zerados")# VOL_FLOORcov_matrix, ajustados = aplicar_vol_floor(cov_matrix, VOL_FLOOR_MENSAL)if ajustados:    print(f"   ⚠️ VOL_FLOOR: {ajustados[:3]}...")# PSDcov_matrix, corrigido = corrigir_psd(cov_matrix)if corrigido:    print("   ⚠️ Matriz PSD corrigida")corr_matrix = retornos_df.corr()print(f"\n✅ Estatísticas calculadas para {NUM_ATIVOS} ativos")

---## ⚙️ BLOCO 4 - OTIMIZAÇÃO

In [ ]:
# =============================================================================# OTIMIZAÇÃO# =============================================================================print("\n" + "=" * 60)print("OTIMIZAÇÃO")print("=" * 60)ret_arr = ret_esperados.valuescov_arr = cov_matrix.valuesprint("\n🔍 GMV...")res_gmv = otimizar_gmv(ret_arr, cov_arr, PESO_MIN_ATIVO, PESO_MAX_ATIVO)ret_gmv, vol_gmv, sharpe_gmv = calcular_metricas(res_gmv.pesos, ret_arr, cov_arr, RF_ANUAL)print("🔍 Max Sharpe...")res_ms = otimizar_max_sharpe(ret_arr, cov_arr, RF_ANUAL, PESO_MIN_ATIVO, PESO_MAX_ATIVO)ret_ms, vol_ms, sharpe_ms = calcular_metricas(res_ms.pesos, ret_arr, cov_arr, RF_ANUAL)print("🔍 Fronteira...")front_vols, front_rets = calcular_fronteira(ret_arr, cov_arr, RF_ANUAL, N_PONTOS_FRONTEIRA, PESO_MIN_ATIVO, PESO_MAX_ATIVO)if TEM_CARTEIRA_ATUAL:    ret_atual, vol_atual, sharpe_atual = calcular_metricas(pesos_carteira, ret_arr, cov_arr, RF_ANUAL)print(f"\n📊 Resultados:")print(f"   GMV:        Ret={ret_gmv*100:.2f}% Vol={vol_gmv*100:.2f}% Sharpe={sharpe_gmv:.3f}")print(f"   Max Sharpe: Ret={ret_ms*100:.2f}% Vol={vol_ms*100:.2f}% Sharpe={sharpe_ms:.3f}")

---## 🎲 BLOCO 5 - MONTE CARLO

In [ ]:
# =============================================================================# MONTE CARLO# =============================================================================print(f"\n🎲 Simulando {NUM_PORTFOLIOS:,} portfólios...")ret_sim, vol_sim, sharpe_sim, pesos_sim = simular_monte_carlo(    ret_arr, cov_arr, RF_ANUAL, NUM_PORTFOLIOS, RANDOM_SEED,    MC_ALPHA_CONC, MC_ALPHA_DIV, MC_FRAC_CANTOS, MC_FRAC_SPARSE,    PESO_MIN_ATIVO, PESO_MAX_ATIVO)print("✅ Simulação concluída!")

---## 📊 BLOCO 6 - VISUALIZAÇÕES

In [ ]:
# =============================================================================# GRÁFICOS# =============================================================================carteiras = {    'GMV': {'ret': ret_gmv, 'vol': vol_gmv, 'sharpe': sharpe_gmv, 'pesos': res_gmv.pesos},    'Max Sharpe': {'ret': ret_ms, 'vol': vol_ms, 'sharpe': sharpe_ms, 'pesos': res_ms.pesos}}if TEM_CARTEIRA_ATUAL:    carteiras['Carteira Atual'] = {'ret': ret_atual, 'vol': vol_atual, 'sharpe': sharpe_atual, 'pesos': pesos_carteira}fig = criar_grafico_fronteira(ret_sim, vol_sim, sharpe_sim, front_vols, front_rets, carteiras, RF_ANUAL, f"Fronteira Eficiente - PyPortfolio {VERSAO}")fig.show()

In [ ]:
fig_corr = criar_heatmap(corr_matrix, "Matriz de Correlação")fig_corr.show()

In [ ]:
fig_pesos = criar_barras_pesos(carteiras, ATIVOS_OTIMIZACAO, TOP_N_PESOS)fig_pesos.show()

---## 📋 BLOCO 7 - TABELAS

In [ ]:
# =============================================================================# TABELA DE MÉTRICAS# =============================================================================print("\n" + "=" * 60)print("TABELA DE MÉTRICAS")print("=" * 60)dados = [    {'Carteira': 'GMV', 'Retorno (%)': ret_gmv*100, 'Vol (%)': vol_gmv*100, 'Sharpe': sharpe_gmv},    {'Carteira': 'Max Sharpe', 'Retorno (%)': ret_ms*100, 'Vol (%)': vol_ms*100, 'Sharpe': sharpe_ms}]if TEM_CARTEIRA_ATUAL:    dados.append({'Carteira': 'Carteira Atual', 'Retorno (%)': ret_atual*100, 'Vol (%)': vol_atual*100, 'Sharpe': sharpe_atual})df_metricas = pd.DataFrame(dados)df_metricas = df_metricas.round({'Retorno (%)': 2, 'Vol (%)': 2, 'Sharpe': 3})display(df_metricas.set_index('Carteira'))

In [ ]:
# =============================================================================# TABELA DE PESOS# =============================================================================print("\n" + "=" * 60)print("TABELA DE PESOS")print("=" * 60)df_pesos = pd.DataFrame({    'Ativo': ATIVOS_OTIMIZACAO,    'GMV (%)': res_gmv.pesos * 100,    'Max Sharpe (%)': res_ms.pesos * 100})if TEM_CARTEIRA_ATUAL:    df_pesos['Carteira Atual (%)'] = pesos_carteira * 100    df_pesos = df_pesos.sort_values('Carteira Atual (%)', ascending=False)else:    df_pesos = df_pesos.sort_values('Max Sharpe (%)', ascending=False)display(df_pesos.head(TOP_N_PESOS).round(2))

---## 📋 BLOCO 8 - RESUMO

In [ ]:
# =============================================================================# RESUMO FINAL# =============================================================================print("\n" + "=" * 70)print(f"RESUMO - PyPortfolio {VERSAO}")print("=" * 70)print(f"\n📊 DADOS")print(f"   Ativos: {NUM_ATIVOS} | Período: {periodo_inicio[:7]} a {periodo_fim[:7]}")print(f"   Ativos USD convertidos: {sum(1 for m in metadados_ativos.values() if m.convertido)}")if TEM_PESOS and cobertura:    print(f"\n📊 COBERTURA")    peso_univ = cobertura['peso_ok'] + cobertura['peso_revisar']    print(f"   No universo: {peso_univ:.1%} | Fora: {1-peso_univ:.1%}")print(f"\n📈 CARTEIRAS")print(f"   GMV:        {ret_gmv*100:>6.2f}% ret | {vol_gmv*100:>6.2f}% vol | {sharpe_gmv:>6.3f} sharpe")print(f"   Max Sharpe: {ret_ms*100:>6.2f}% ret | {vol_ms*100:>6.2f}% vol | {sharpe_ms:>6.3f} sharpe")if TEM_CARTEIRA_ATUAL:    print(f"   Atual:      {ret_atual*100:>6.2f}% ret | {vol_atual*100:>6.2f}% vol | {sharpe_atual:>6.3f} sharpe")print("\n" + "=" * 70)print("✅ Análise concluída!")

---## 🗺️ Roadmap (Próximas Versões) — Priorizado### 1. 🏷️ Restrições por Classe + Max por Ativo — Alta Prioridade**Por que importa:** Carteiras práticas têm limites (máx 10% cripto, mín 20% RF, etc.)**Complexidade:** Média**Dependências:** Classificação de ativos por tipo**Implementação:**- Adicionar metadado `classe` por ativo- Bounds por classe no otimizador- Relatório de alocação por classe---### 2. 🛡️ Covariância Robusta (Ledoit-Wolf / Shrinkage) — Alta Prioridade**Por que importa:** Com poucos dados, covariância amostral é instável**Complexidade:** Média**Dependências:** Nenhuma**Implementação:**- sklearn.covariance.LedoitWolf- Opção EWMA (decay factor)- Comparativo: sample vs shrunk---### 3. 📊 Backtesting Out-of-Sample — Média Prioridade**Por que importa:** Validar se a otimização "funciona" fora da amostra**Complexidade:** Alta**Dependências:** [1] Restrições por classe**Implementação:**- Split train/test- Walk-forward com rebalanceamento- Métricas: Sharpe realizado, drawdown, turnover---### 4. 📈 Black-Litterman — Baixa Prioridade**Por que importa:** Incorporar views do investidor de forma sistemática**Complexidade:** Alta**Dependências:** [1] e [2]**Implementação:**- Prior de mercado (pesos de capitalização)- Views (absolutos e relativos)- Posterior ajustado---### 5. 🖥️ Export e UX — Baixa Prioridade**Por que importa:** Facilitar uso e compartilhamento**Complexidade:** Baixa**Implementação:**- Export CSV/HTML/Excel- Filtros interativos- Themes customizados

---## ✅ Checklist de Validação (v2.7)### Conversão USD→BRL- [ ] Ativos USD são identificados corretamente- [ ] Série de câmbio é baixada do yfinance- [ ] Fórmula r_BRL = (1+r_USD)*(1+r_FX)-1 é aplicada- [ ] Validação mostra cálculo correto para 5 linhas### Cobertura de Pesos- [ ] Auditoria mostra % OK, REVISAR, excluído, não mapeado- [ ] Política "reportar"/"renormalizar"/"caixa_cdi" funciona- [ ] Nunca remapeia para ativo errado### Monte Carlo- [ ] Respeita peso_max_ativo- [ ] Gera cantos, sparse, concentrado, diversificado### Rodar do Zero- [ ] Com Excel + pesos: funciona- [ ] Com Excel sem pesos: funciona- [ ] Sem Excel (yfinance): funciona (se habilitado)- [ ] Com ativos USD + BRL juntos: conversão correta